<a href="https://www.kaggle.com/code/dietzschenostoevsky/mnist-svm-augmented-data?scriptVersionId=126995301" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Got 0.98582 score. Using non Deep Learning method 

## SVM Classifier with Augmented data : should be 98% accurate 

In [ ]:
import numpy as np
import pandas as pd
import sklearn

In [ ]:
train = pd.read_csv(r"/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv(r"/kaggle/input/digit-recognizer/test.csv")

In [ ]:
X_np = train.drop(['label'], axis=1).to_numpy()
y_np = train['label'].to_numpy()
test_np = test.to_numpy()
print("Total Images :", X_np.shape[0])
print("Total Labels: ",y_np.shape[0])
print("Test Set Images: ",test_np.shape[0])

In [ ]:
from scipy.ndimage import shift
def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])

In [ ]:
X_augmented = [image for image in X_np]
y_augmented = [label for label in y_np]

for dx, dy in ((-1,0), (1,0), (0,1), (0,-1)):
    for image, label in zip(X_np, y_np):
        X_augmented.append(shift_image(image, dx, dy))
        y_augmented.append(label)

X_augmented = np.array(X_augmented)
y_augmented = np.array(y_augmented)
print(F"Total augmented images = {len(X_augmented)}")        

In [ ]:
shuffle_idx = np.random.permutation(len(X_augmented))
X_augmented = X_augmented[shuffle_idx]
y_augmented = y_augmented[shuffle_idx]

In [ ]:
import time
t3 = time.time()
from sklearn.svm import SVC

svm_clf_aug = SVC(random_state=73, verbose=True)
svm_clf_aug.fit(X_augmented, y_augmented)

print(F"took {time.time() - t3:.2f} secs to train")

In [ ]:
# Submission file
import pandas as pd
print("Running prediction")
y_pred = svm_clf_aug.predict(test_np)
submission_df = pd.DataFrame({'ImageId': range(1, len(y_pred)+1), 'Label': y_pred})
submission_df.to_csv('submission.csv', index=False)
print("Submission done")